In [26]:
from langchain_community.document_loaders import DirectoryLoader
import streamlit as st




In [ ]:
loader = DirectoryLoader("resources/data/finance")
docs = loader.load()
print(len(docs))

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


2


In [28]:
print(docs[0].page_content[:1000])

Financial Report for FinSolve Technologies Inc. - 2024

Executive Summary:

2024 marked a year of both opportunity and challenge for FinSolve Technologies. Despite a robust revenue increase, we saw significant pressure in certain expense categories, notably vendor-related costs and software subscriptions. However, these pressures were balanced by cost-saving measures in operational efficiency, strong gross margin performance, and strategic investment in growth areas. The company is well-positioned to continue scaling its core offerings, but focused attention on cost optimization will be essential for maintaining profitability in the coming years.

Year-Over-Year (YoY) Analysis:

FinSolve Technologies's revenue grew by 25% in 2024, driven largely by the global expansion of its services, especially in Asia and Europe. This was accompanied by a 10% increase in vendor-related expenses, impacting overall profit margins. While gross profit increased by 25%, reflecting higher operational effi

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)
documents[:5]

[Document(metadata={'source': 'resources\\data\\finance\\financial_summary.md'}, page_content='Financial Report for FinSolve Technologies Inc. - 2024\n\nExecutive Summary:\n\n2024 marked a year of both opportunity and challenge for FinSolve Technologies. Despite a robust revenue increase, we saw significant pressure in certain expense categories, notably vendor-related costs and software subscriptions. However, these pressures were balanced by cost-saving measures in operational efficiency, strong gross margin performance, and strategic investment in growth areas. The company is well-positioned to continue scaling its core offerings, but focused attention on cost optimization will be essential for maintaining profitability in the coming years.\n\nYear-Over-Year (YoY) Analysis:'),
 Document(metadata={'source': 'resources\\data\\finance\\financial_summary.md'}, page_content="Year-Over-Year (YoY) Analysis:\n\nFinSolve Technologies's revenue grew by 25% in 2024, driven largely by the globa

In [30]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents[:],OllamaEmbeddings())

In [31]:
query = "what's the scope"
result = db.similarity_search(query)
result[0].page_content

'Post-incident analysis and lessons learned\n\n5.3.3 Security Training\n\nMandatory security awareness training for all employees\n\nRole-specific security training for developers, administrators\n\nQuarterly phishing simulations\n\nSecurity champions program within engineering teams\n\n6. Testing and Quality Assurance\n\n6.1 Testing Strategy\n\n6.1.1 Test Pyramid\n\nUnit Tests:\n\nCover 90% of code base\n\nFocus on business logic and edge cases\n\nImplemented using Jest (Node.js), Pytest (Python), Go testing\n\nIntegration Tests:\n\nValidate microservice interactions\n\nTest database operations and external service integrations\n\nImplemented using Postman/Newman and custom test harnesses\n\nEnd-to-End Tests:\n\nSimulate complete user journeys\n\nCover critical business flows\n\nImplemented with Cypress (web) and Appium (mobile)\n\n6.1.2 Specialized Testing\n\nPerformance Testing:\n\nLoad testing with JMeter (target: 2,000 concurrent users)\n\nStress testing to identify breaking point

In [32]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama2")
llm

OllamaLLM(model='llama2')

In [33]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    
    """
    You are a helpful assistant who helps users to search all <context> you have and provide answer to them and addtionally provide them the link to that file where you have fetched the answer from.
    <context>
    {context}
    </context>

    input: {input}
    """
)

In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)

In [35]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018BC9F9D540>, search_kwargs={})

In [36]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [41]:
response = retrieval_chain.invoke({"input": "Whats the Q3 - July to September 2024 Operating Income"})
print(response['answer'])

The operating income for Q3 of 2024 is $475 million, as shown in the table below:

| Line Item | Amount ($Million) |
| --- | --- |
| Revenue | $2,750 |
| Cost of Goods Sold | $1,050 |
| Gross Profit | $1,700 |
| Operating Expenses | $950 |
| Operating Income | $475 |

Note: The above figures are hypothetical and used for illustration purposes only.


In [105]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_community.document_loaders import TextLoader

# Don’t use mode="elements"
loader = DirectoryLoader(
    "resources/data/engineering",
    glob="**/*.md",
    loader_cls=TextLoader
)
docs = loader.load()
print(docs)

[Document(metadata={'source': 'resources\\data\\engineering\\engineering_master_doc.md'}, page_content='# FinSolve Technologies Engineering Document\n\n## 1. Introduction\n\n### 1.1 Company Overview\nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.\n\n### 1.2 Purpose\nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve\'s mission: "To empower financial freedom through secure, scalable, and innovative technology solutions."\n\n### 1.3 Scope

In [106]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
    ("#", "h1"),
    ("##", "h2"),
    ("###", "h3"),
])

# Combine markdown strings if needed
combined_md = "\n\n".join([doc.page_content for doc in docs])
chunks = splitter.split_text(combined_md)

for c in chunks:
    print(c.metadata)
    print(c.page_content[:])


{'h1': 'FinSolve Technologies Engineering Document', 'h2': '1. Introduction', 'h3': '1.1 Company Overview'}
FinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.
{'h1': 'FinSolve Technologies Engineering Document', 'h2': '1. Introduction', 'h3': '1.2 Purpose'}
This engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve's product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve's mission: "To empower financial freedom through secure, scalable, and innovative technology solutions."
{'h1': 'FinSolve Technologies E

In [107]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks_before_header = recursive_splitter.split_documents(chunks)
chunks_before_header

[Document(metadata={'h1': 'FinSolve Technologies Engineering Document', 'h2': '1. Introduction', 'h3': '1.1 Company Overview'}, page_content='FinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.'),
 Document(metadata={'h1': 'FinSolve Technologies Engineering Document', 'h2': '1. Introduction', 'h3': '1.2 Purpose'}, page_content='This engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a comprehensive guide for engineering teams, stakeholders, and partners to ensure alignment with FinSolve\'s mission: "To empower financial freedom through secure, scalab

In [114]:
from langchain.schema import Document

# Append structured headings to each chunk after splitting
final_chunks = []
for chunk in chunks_before_header:  # Output of recursive_splitter.split_documents(...)
    h1 = chunk.metadata.get("h1", "")
    h2 = chunk.metadata.get("h2", "")
    h3 = chunk.metadata.get("h3", "")

    # Build a structured markdown header (or plain text if you prefer)
    header = ""
    if h1: header += f"{h1}\n"
    if h2: header += f"{h2}\n"
    if h3: header += f"{h3}\n"
    header = header.strip()

    # Combine header and content
    new_content = f"{header}\n\n{chunk.page_content}" if header else chunk.page_content

    final_chunks.append(Document(page_content=new_content, metadata=chunk.metadata))
final_chunks

[Document(metadata={'h1': 'FinSolve Technologies Engineering Document', 'h2': '1. Introduction', 'h3': '1.1 Company Overview'}, page_content='FinSolve Technologies Engineering Document\n1. Introduction\n1.1 Company Overview\n\nFinSolve Technologies is a leading FinTech company headquartered in Bangalore, India, with operations across North America, Europe, and Asia-Pacific. Founded in 2018, FinSolve provides innovative financial solutions, including digital banking, payment processing, wealth management, and enterprise financial analytics, serving over 2 million individual users and 10,000 businesses globally.'),
 Document(metadata={'h1': 'FinSolve Technologies Engineering Document', 'h2': '1. Introduction', 'h3': '1.2 Purpose'}, page_content='FinSolve Technologies Engineering Document\n1. Introduction\n1.2 Purpose\n\nThis engineering document outlines the technical architecture, development processes, and operational guidelines for FinSolve\'s product ecosystem. It serves as a compreh

In [115]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
md_db = Chroma.from_documents(final_chunks[:],OllamaEmbeddings())

In [116]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2")
llm

OllamaLLM(model='llama3.2')

In [117]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    
  """
    You are an expert assistant with access to internal company documents. Answer the user's question with absolute clarity using only the information from <context>. Do not guess or make assumptions. Prioritize accuracy and relevance.
    
    <context>
    {context}
    </context>

    User Question: {input}

    Answer in a confident and concise tone, as if you're briefing a decision-maker.
    INSTRUCTIONS:
    1. Each chunk starts with:
     - Main Heading
     - Side Heading
     - Sub Sub Heading
    2. Dont make your own asssumtions only mention if its present in the <context>
    """
)

In [118]:
from langchain.chains.combine_documents import create_stuff_documents_chain
md_document_chain = create_stuff_documents_chain(llm,prompt)

In [119]:
md_retriever = md_db.as_retriever(search_kwargs={"k": 10})
md_retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001C3D8A19990>, search_kwargs={'k': 10})

In [120]:
retrieved_docs = md_retriever.get_relevant_documents("What's the Q1 - January to March 2024 Quarterly Financial Overview")

for i, doc in enumerate(retrieved_docs):
    print(f"\n---- Chunk {i} ----")
    print(doc.page_content[:300])
    print("H2:", doc.metadata.get("h2", ""))
    print("H3:", doc.metadata.get("h3", ""))



---- Chunk 0 ----
FinSolve Technologies Engineering Document
6. Testing and Quality Assurance
6.3 Defect Management

#### 6.3.1 Bug Tracking
* Jira for logging and tracking defects
* Required fields: steps to reproduce, expected vs. actual results, environment
* Severity classification:
* S1 (Critical): System unusab
H2: 6. Testing and Quality Assurance
H3: 6.3 Defect Management

---- Chunk 1 ----
FinSolve Technologies Engineering Document
10. Appendices
10.2 Reference Documents

| Document | Location | Purpose |
|----------|----------|---------|
| AWS Well-Architected Framework | Internal Wiki | Cloud architecture best practices |
| PCI-DSS Guidelines | Security Portal | Payment security com
H2: 10. Appendices
H3: 10.2 Reference Documents

---- Chunk 2 ----
2024 Annual Summary

Cash flow from operations reached $1.5 billion, up 14% YoY, enabling $260 million in investments for R&D, market expansion, and product development. Financing activities provided $110 million to support workin

In [121]:
from langchain.chains import create_retrieval_chain
md_retrieval_chain = create_retrieval_chain(md_retriever,md_document_chain)

In [122]:
response = md_retrieval_chain.invoke({"input": "What's the Q1 - January to March 2024 Quarterly Financial Overview"})
print(response['answer'])

**Q1 - January to March 2024 Quarterly Financial Overview**

### Executive Summary

The Q1 financial overview for FinSolve Technologies Inc. 2024 is available.

### Expense Breakdown

Expenses supported retention and expansion strategies:  
- **Vendor Services**: $110 million, with 30% ($33 million) allocated to loyalty programs and customer retention strategies.
- **Employee Benefits & HR**: $40 million, reflecting increased recruitment and training costs to support growth.
- **Software Subscriptions**: $40 million, maintaining investment in cloud-based infrastructure.
- **Other Operational Expenses**: $25 million, covering events, merchant partnerships, and regional activities.

### Quarterly Financial Report - FinSolve Technologies Inc. 2024

The report is available for review, providing detailed financial data and analysis of the company's performance during Q1 2024.
